# SMD Python Hands On

In [ ]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

## Inhalt
<div id="toc"></div>

## Python 

### Warum python?

<img alt="stackoverflow" src="https://zgab33vy595fw5zq-zippykid.netdna-ssl.com/wp-content/uploads/2017/09/growth_major_languages-1-1024x878.png" width="600px" />


## Grundlagen

Werden heute hier nicht besprochen, gute Einstiegspunkte sind:

* [PeP et al. Toolbox Workshop](https://toolbox.pep-dortmund.org)
* [The scientific Python lectures](https://github.com/jrjohansson/scientific-python-lectures)
* [A Byte Of Python](https://python.swaroopch.com/)

Insbesondere für den Hauptteil der Vorlesung:
* [The Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/)



## Style Guide und Linter

* Für python gibt es einen allgemein annerkannte Style Guide, pep8. Beachten!

* `conda install flake8 pyflakes pycodestyle`

    * `pycodestyle test.py`
    * `pyflakes test.py`
    * `flake8 test.py`

* Alle vernünftigen Editoren können Fehler und Nichtbefolgung eines Style Guides markieren (Linting).
  * VS Code Plugin: https://marketplace.visualstudio.com/items?itemName=ms-python.python
  * Atom-Plugin: https://atom.io/packages/linter-flake8
  * Vim-Plugins: https://github.com/neomake/neomake oder https://github.com/vim-syntastic/syntastic


## numpy

Wiederholdung der wichtigsten numpy Eigenschaften.

* Schnell durch Vektorisierung und kompilierten C++/C/Fortran-Code  
   ⇒ Keine Python Schleifen über numpy arrays
* Viele Funktionen für Datenanalyse, Zufallszahlen, Numerik, Lineare Algebra etc


In [ ]:
import numpy as np

### Indizierung & Masken

Indizierung über Slices und/oder boolean Masken

In [ ]:
a = np.random.normal(0, 1, 10)
a

In [ ]:
a > 0

In [ ]:
a[a > 0]

In [ ]:
# Klammern wichtig
# | = or
# & = and
# ~ = not

a[(a > -1) & (a < 1)]

In [ ]:
np.logical_and(a > -1, a < 1)

### Das axis keyword

Wichtig für aggregierende Operationen (z.B. `np.sum`, `np.mean`, `np.prod`, `np.std`)

In [ ]:
X = np.arange(12).reshape(4, 3)
X

In [ ]:
np.sum(X)

In [ ]:
np.sum(X, axis=0)

### Broadcasting

Arrays verschiedener Größe miteinander verrechnen, wo es geht

In [ ]:
a = np.arange(12).reshape(4, 3)
b = 5
c = np.arange(3)
d = np.arange(4)

In [ ]:
a - b

In [ ]:
a - c

In [ ]:
# a - d -> error
(a.T - d).T

### Timing Beispiel: Loops vs. Numpy vektorisiert

Aufgabe: Rechne den Punkt mit dem kleinsten Abstand zu einem anderen Punkt aus

In [ ]:
point = (0, 1)
points = [
    (0, 0),
    (0.5, -0.5),
    (1, -1),
    (0, 2),
    (0, 1.1),
    (-2, 3),
    (5, 1),
    (10, 4),
    (-4, 2),
    (-3, 0),
] * 100

Pure Python Lösung mit loops:

In [ ]:
def find_closest(points, point):
    min_distance = float('inf')
    for i, other in enumerate(points):
        distance = ((other[0] - point[0])**2 + (other[1] - point[1])**2)**0.5
        if distance < min_distance:
            min_distance = distance
            min_idx = i
    
    return min_idx

idx = find_closest(points, point)
print(idx, points[idx])

In [ ]:
%%timeit 
find_closest(points, point)

In [ ]:
points = np.array(points)
point = np.array(point)

In [ ]:
def find_closest_numpy(points, point):
    distances = np.linalg.norm(points - point, axis=1)
    idx = np.argmin(distances)
    return idx

idx = find_closest_numpy(points, point)
print(idx, points[idx])

In [ ]:
%%timeit 
find_closest_numpy(points, point)

### Random numbers

In [ ]:
uniform = np.random.uniform(-5, 5, 1000)
gaussian = np.random.normal(0, 1, 1000)
poisson = np.random.poisson(3, 1000)

mean = [2, 1]
cov = [[2, 1],
       [1, 4]]
gauss_2d = np.random.multivariate_normal(mean, cov, 1000)

Für reproduzierbarkeit wichtig: der Startwert (seed) des Zufallszahlengenerators

In [ ]:
np.random.seed(42)

np.random.normal()

## matplotlib

Die zwei wichtigsten Plot-Arten für die Datenanalyse sind Histogramme und Streudiagramme

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

plt.rcParams.update({'figure.figsize': (12, 8), 'font.size': 12})


In [ ]:
plt.hist(gaussian, bins=20, range=[-5, 5])
None

Bei Vergleichen ist es wichtig, das gleiche Binning zu verwenden

In [ ]:
plt.hist(gaussian, bins=20, range=[-5, 5], histtype='step', label='Gaussian')
plt.hist(uniform, bins=20, range=[-5, 5], histtype='step', label='Uniform')
plt.legend()
None

### Bei diskreten Werten unbedingt ganzzahlige, zentrierte bins verwenden

In [ ]:
plt.hist(poisson, bins=15)
None

In [ ]:
plt.hist(poisson, bins=np.arange(15) - 0.5, edgecolor='w') # bins kann entweder Anzahl oder Grenzen sein

None

In [ ]:
plt.scatter(gauss_2d[:, 0], gauss_2d[:, 1])

In [ ]:
gauss_2d = np.random.multivariate_normal(mean, cov, 10000)
plt.scatter(gauss_2d[:, 0], gauss_2d[:, 1], s=5, alpha=0.2)

* Scatter kann ein Drittes Attribut als Farbe darstellen
* Mit ListedColorMap bekommt man eine diskrete Colormap mit einer Farbe für jede Klasse

In [ ]:
from sklearn.datasets import load_iris

iris = load_iris()

In [ ]:
from matplotlib.colors import ListedColormap

# ColorMap mit N diskreten Farben aus der
# matplotlib Farb rotation, 'C0', 'C1', ... 
cmap = ListedColormap([
    'C{}'.format(i) 
    for i in range(len(iris.target_names))
])

plt.scatter(
    iris.data[:, 0],  # x-werte, erste Spalte
    iris.data[:, 1],  # y-werte, zweite Spalte 
    c=iris.target,    # Farbachse, Sorte
    cmap=cmap,        # colormap, siehe oben
    vmin=-0.5,        # Minimum der Farbachse
    vmax=len(iris.target_names) - 0.5,  # Maximimum der Farbachse
)


plt.xlabel(iris.feature_names[0])
plt.ylabel(iris.feature_names[1])

bar = plt.colorbar(ticks=[0, 1, 2])
bar.set_ticklabels(iris.target_names)

In [ ]:
plt.hist2d(
    gauss_2d[:, 0],
    gauss_2d[:, 1],
    bins=[20, 20],
    range=[[-3, 7], [-7, 9]]
)
None

Manchmal hilfreich: Logarithmische Skala auf der Farbachse

z.B. um eine zweite, kleine Population zu erkennen

In [ ]:
gauss_2d = np.random.multivariate_normal(mean, cov, 100000)
gauss_2d_2 = np.random.multivariate_normal([-1.5, 4.5], [[0.5, 0], [0, 0.5]], 500)

gauss_2d_both = np.concatenate([gauss_2d, gauss_2d_2], axis=0)

gauss_2d_both.shape

In [ ]:
plt.hist2d(
    gauss_2d_both[:, 0],
    gauss_2d_both[:, 1],
    bins=[50, 50],
    range=[[-3, 7], [-7, 9]]
)
plt.colorbar()

In [ ]:
from matplotlib.colors import LogNorm

plt.hist2d(
    gauss_2d_both[:, 0],
    gauss_2d_both[:, 1],
    bins=[50, 50],
    range=[[-3, 7], [-7, 9]],
    norm=LogNorm(),
)
plt.colorbar()

## scipy

Für SMD werden vor allem das `scipy.optimize` und das `scipy.stats` Modul wichtig werden

### scipy.stats

Viele Statistische Verteilungen, Likelihood-Fits eingebaut

In [ ]:
from scipy.stats import norm

mean = 5
std = 2

x = np.linspace(mean - 5 * std, mean + 5 * std, 250)

gaussian = norm(mean, std)
samples = gaussian.rvs(size=1000)

plt.plot(x, gaussian.pdf(x), label='Wahrscheinlichkeitsdichte')
plt.plot(x, gaussian.cdf(x), label='Verteilungsfunktion')


# In matplotlib < 2.1 hieß `density` `normed`
plt.hist(
    samples,
    bins=100,
    range=[x.min(), x.max()],
    density=True,  # Fläche des Histogramms soll 1 sein
    label='Normiertes Histogramm',
)

plt.legend()

In [ ]:
x = norm(5, 1).rvs(100)

norm.fit(x)

### scipy.optimize

Ihr kennt alle `curve_fit` aus dem Praktikum, `scipy` kann mehr:
* Nullstellen finden
* Allgemeine Funktionen minimiere (mit Randbedingungen)
* (Unterbestimmte) Gleichungssysteme Lösen

#### Nullstellen & Minima

Genauere Informationen siehe Docs der einzelnen Funktionen

In [ ]:
from scipy.optimize import newton, brentq, minimize

In [ ]:
def f(x):
    return (x - 5)**2 - 1

In [ ]:
# Nullstellen

x0_1 = newton(f, 1)
x0_2 = newton(f, 8)

x0_1, x0_2

In [ ]:
# Minimum

result = minimize(f, x0=(2))
result

In [ ]:
minimum = result.x
minimum

In [ ]:
x = np.linspace(2, 8, 250)
plt.plot(x, f(x))
plt.grid()

plt.axvline(x0_1, color='C1', label='Nullstelle 1')
plt.axvline(x0_2, color='C2', label='Nullstelle 2')
plt.axvline(minimum, color='C3', label='Minimum')

plt.legend()

#### Likelihood-Fit (Mehr dazu später in der Vorlesung)

Beispiel Higgs Messung

Wir haben Normal-Verteiltes Signal der Higgs-Masse und Exponentiell verteilten Untergrund und möchten
die Higgs-Masse mit einem Fit bestimmen

In [ ]:
np.random.seed(42)

e_min = 75
e_max = 175

higgs_signal = np.random.normal(126, 5, 500)
background = np.random.exponential(50, size=20000)
background = background[(background > e_min) & (background < e_max)]

measured = np.append(higgs_signal, background)



plt.hist(measured, bins=100)
plt.xlabel('$m / \mathrm{GeV}$')
None

In [ ]:
from scipy.stats import norm, expon


def pdf(x, mean, std, tau, p, e_min, e_max):
    N = np.exp(-e_min / tau) - np.exp(-e_max / tau)
    return (
        p * norm.pdf(x, mean, std) 
        + (1 - p) / N * expon.pdf(x, scale=tau)
    )

def neg_log_likelihood(params, data, e_min, e_max):
    return -np.sum(np.log(pdf(data, *params, e_min=e_min, e_max=e_max)))

In [ ]:
result = minimize(
    neg_log_likelihood,
    x0=(130, 2, 30, 0.2),
    bounds=[
        (None, None), # No bounds for mean
        (1e-30, None), # std > 0
        (1e-30, None), # tau > 0
        (0, 1), # 0 <= p <= 1
    ],
    args=(measured, e_min, e_max)
)

print(result.x)

In [ ]:
print('Higgs mass is {} ± {} GeV'.format(result.x[0], np.sqrt(result.hess_inv.todense()[0, 0])))

In [ ]:


cov = result.hess_inv.todense()
A = np.diag(1 / np.sqrt(np.diag(cov)))

cor = A @ cov @ A.T

plt.matshow(cor, cmap='RdBu_r', vmin=-1, vmax=1)
plt.xticks(np.arange(4), ['mean', 'std', 'tau', 'p'])
plt.yticks(np.arange(4), ['mean', 'std', 'tau', 'p'])
plt.colorbar()

In [ ]:
hist, edges, plot = plt.hist(measured, bins=100)
m = np.linspace(e_min, e_max, 250)
plt.plot(m, pdf(m, *result.x, e_min, e_max) * np.diff(edges)[0] * len(measured))

## pandas

Docu: [Hier](https://pandas.pydata.org/pandas-docs/stable/)

Bibliothek für Datenanalyse, zentrales Konzept: `pd.DataFrame` → 2d-Tabelle aus Daten

In [ ]:
import pandas as pd

In [ ]:
signal = pd.DataFrame({
    'x': np.random.normal(0, 1, 1000),
    'y': np.random.normal(0, 1, 1000),
    'N': np.random.poisson(20, 1000),
    't': np.random.exponential(5, 1000),
})

In [ ]:
signal.plot.scatter('x', 'y')

In [ ]:
signal['t'].plot.hist()

### Input/Output

In [ ]:
signal.to_csv('data.csv', index=False)

In [ ]:
signal = pd.read_csv('data.csv')

In [ ]:
# HDF5 is a fast, binary format
signal.to_hdf('data.hdf5', key='signal')

In [ ]:
background = pd.DataFrame({
    'x': np.random.uniform(-4, 4, 10000),
    'y': np.random.uniform(-4, 4, 10000),
    'N': np.random.poisson(30, 10000),
    't': np.random.exponential(10, 10000),
})

In [ ]:
# Mehrere DataFrames in einer Datei speichern
background.to_hdf('data.hdf5', key='background')

In [ ]:
background.head()

In [ ]:
df = pd.read_hdf('data.hdf5', key='signal')

In [ ]:
len(df)

Look at the first / last values

In [ ]:
df.head()

In [ ]:
df.describe()

### The titanic dataset

In [ ]:
df = pd.read_csv('titanic.csv')

In [ ]:
df.head()

Wie viele Valide Werte gibt es in jeder Spalte?

In [ ]:
df.count()

Spalten loswerden, die zu viele missing values haben

In [ ]:
# axis=1 Spalten droppen
# inplace=df direkt bearbeiten
df.drop(['cabin', 'boat', 'body', 'home.dest'], axis=1, inplace=True) 

df.head()

Wie war die Geschlechter Verteilung auf der Titanic?

In [ ]:
df.sex.value_counts()

Mächtige Operation: GroupBy → Aggregate

Datensatz in mehrere Gruppen unterteilen und pro Gruppe zusammenfassen.

Hier: Aufgeschlüsselt nach Geschlecht, den Prozentsatz der überlebenden.

In [ ]:
df.groupby('sex')['survived'].agg('mean')

Auch DataFrames unterstützen masken:

In [ ]:
df['child'] = df.age < 9

df[df.child].survived.mean(), df[~df.child].survived.mean()

Alternativ

In [ ]:
df.groupby('child').survived.mean()

## joblib

Einfaches tool um Sachen Parallel zu machen

In [ ]:
from joblib import delayed, Parallel

In [ ]:
from time import sleep
import random

def long_running_calculation(num):
    sleep(random.random())
    return num**2


with Parallel(n_jobs=4, verbose=10) as pool:
    result = pool(delayed(long_running_calculation)(i) for i in range(100))

print(result)